## Backtesting value investing ##

In [1]:
import pandas as pd
from pandasql import sqldf
import sqlalchemy
import numpy as np
# Helper function to run SQL queries on DataFrames
def sql(q): 
    return sqldf(q, globals())
import os
import itertools
import plotly

In [2]:
df = pd.read_csv('results.csv')

In [3]:
# Example 1: Get the first's quarter record for each ticker
query1 = """
SELECT 
    ticker,
    date,
    eps,
    per,
    ebitda,
    pbv,
    solvency,
    roe,
    fcf,
    shares,
    prices
FROM df
"""

In [4]:
data = sql(query1)

In [5]:
data.head(2)

,ticker,date,eps,per,ebitda,pbv,solvency,roe,fcf,shares,prices
0,AAPL,2022-09-30,6.259821838783072,21.734218824498264,130541000000.0,42.80747239780155,2.290884648855061,196.95887275023682,-65518000000.0,15943425000.0,136.05233764648438
1,AAPL,2023-09-30,6.237596109751595,27.58509022734883,125820000000.0,43.05370943587197,2.426452776171993,156.07601454639075,-85213000000.0,15550061000.0,172.0646514892578


In [6]:
#cleaning 
data = data[data.eps != 'eps']
data = data[data.eps.notna()]
data = data[data.per.notna()]
data = data[data.ebitda.notna()]
data = data[data.pbv.notna()]
data = data[data.solvency.notna()]
data = data[data.roe.notna()]
data = data[data.fcf.notna()]
data = data[data.prices.notna()]
data.reset_index(inplace=True)
data.eps = data.eps.astype(float)
data.per = data.per.astype(float)
data.ebitda = data.ebitda.astype(float)
data.pbv = data.pbv.astype(float)
data.solvency = data.solvency.astype(float)
data.roe = data.roe.astype(float)
data.fcf = data.fcf.astype(float)
data.shares = data.shares.astype(float)
data.prices = data.prices.astype(float)
data.sort_values(by=['ticker', 'date'], ascending=True, inplace=True)

In [7]:
#Add 2-quarter returns
data['2qf_price'] = data.groupby('ticker')['prices'].shift(-2)
data['2qf_returns'] = data['2qf_price'] / data['prices'] - 1

In [8]:
#Getting first quarter record for each ticker
firstq = data.groupby('ticker').first()

In [9]:
firstq.head(3)

,index,date,eps,per,ebitda,pbv,solvency,roe,fcf,shares,prices,2qf_price,2qf_returns
ticker,,,,,,,,,,,,,
A,938,2022-10-31,4.250847,31.842172,1.905000e+09,7.526877,5.659323,23.638077,-7.042000e+09,2.950000e+08,135.356216,129.261505,-0.045027
AAPL,0,2022-09-30,6.259822,21.734219,1.305410e+11,42.807472,2.290885,196.958873,-6.551800e+10,1.594342e+10,136.052338,231.920639,0.704643
ABBV,99,2022-12-31,6.659578,21.845802,2.417400e+10,14.917540,4.699201,68.285615,-7.585700e+10,1.769181e+09,145.483810,171.696518,0.180176


In [10]:
#preprocessing

results = firstq.reset_index()

#market cap for normalization
results['market_cap'] = results['shares'] * results['prices']

# --- Normalize metrics by size/price --- to compare companies of different sizes
results['eps_norm'] = results['eps'] / results['prices']             # earnings yield
results['ebitda_norm'] = results['ebitda'] / results['market_cap']          # EBITDA/MarketCap
results['fcf_norm'] = results['fcf'] / results['market_cap']                # FCF/MarketCap

# This metrcis are already ratio metrics
results['per'] = results['per']
results['pbv'] = results['pbv']
results['solvency'] = results['solvency']
results['roe'] = results['roe']

#We'll use z-score normalization as an alternative method to ranks
def zscore(series):
    return (series - series.mean()) / (series.std(ddof=0) if series.std(ddof=0) != 0 else 1)

# Compute z-scores for each metric
results['z_eps'] = zscore(results['eps_norm'])
results['z_per'] = -1*zscore(results['per'])
results['z_ebitda'] = zscore(results['ebitda_norm'])
results['z_pbv'] = -1*zscore(results['pbv'])
results['z_solvency'] = zscore(results['solvency'])
results['z_roe'] = zscore(results['roe'])
results['z_fcf'] = zscore(results['fcf_norm'])

## Testing with manual approach ##

In [11]:
results['avg_zscore'] = (
    results['z_eps'] +
    results['z_per'] +
    results['z_ebitda'] +
    results['z_pbv'] +
    results['z_solvency'] +
    results['z_roe'] +
    results['z_fcf']
) / 7

# --- Rank companies by z-score (higher = better fundamentals) ---
results['avg_zscore_rank'] = results['avg_zscore'].rank(ascending=False)

# --- Sort final results by z-score rank (best to worst) ---
results = results.sort_values(by='avg_zscore_rank', ascending=True)

results['market_cap_percentile'] = results['market_cap'].rank(pct=True)

In [12]:
low_cap_stocks = results[results['market_cap_percentile'] <= .15]
low_cap_stocks = low_cap_stocks[low_cap_stocks['avg_zscore_rank'] <= 50]
top_low_cap_stocks = low_cap_stocks.sort_values(by='avg_zscore_rank', ascending=True).head(15)
top_low_cap_stocks.head(1)

,ticker,index,date,eps,per,ebitda,pbv,solvency,roe,fcf,...,z_eps,z_per,z_ebitda,z_pbv,z_solvency,z_roe,z_fcf,avg_zscore,avg_zscore_rank,market_cap_percentile
50,BLDR,1551,2022-12-31,19.799005,3.300671,4.267346e+09,1.828643,5.749555,55.402165,-4.656010e+09,...,3.91138,0.18976,4.383492,0.09797,-0.180259,0.132754,0.164892,1.242856,2.0,0.032787


In [13]:
def calculate_sortino_ratio(returns, risk_free_rate=0.05, periods_per_year=2):
    
    # Convert returns to decimal
    returns_decimal = np.array(returns) / 100
    
    # Calculate excess returns
    excess_returns = returns_decimal - (risk_free_rate / periods_per_year)
    
    # Calculate downside returns (returns below target/risk-free rate)
    downside_returns = np.where(returns_decimal < (risk_free_rate / periods_per_year), 
                               returns_decimal - (risk_free_rate / periods_per_year), 
                               0)
    
    # Calculate downside deviation
    downside_deviation = np.sqrt(np.mean(np.square(downside_returns))) * np.sqrt(periods_per_year)
    
    # Calculate Sortino ratio
    sortino_ratio = (np.mean(excess_returns) * periods_per_year) / downside_deviation if downside_deviation != 0 else 0
    
    return sortino_ratio

In [14]:
def strategy_backtest(data, market_cap_percentile, initial_investment, split_method, stocks_to_invest, min_zscore_rank, experiment_id):

    # total portfolio tracking
    portfolio_list = []
    total_invested = 0
    total_final_value = 0
    remaining_cash = initial_investment
    
    # Filter for small-cap stocks and get top N by z-score(STRATEGY)
    low_cap_stocks = data[data['market_cap_percentile'] <= market_cap_percentile]
    low_cap_stocks = low_cap_stocks[low_cap_stocks['avg_zscore_rank'] <= min_zscore_rank]
    top_low_cap_stocks = low_cap_stocks.sort_values(by='avg_zscore_rank', ascending=True).head(stocks_to_invest)
    
    if split_method == 'equal':
        target_investment_per_stock = initial_investment / len(top_low_cap_stocks)
        
        for _, stock in top_low_cap_stocks.iterrows():
            initial_price = stock['prices']  # Current price
            final_price = stock['2qf_price']  # Price after 2 quarters
            
            # Calculate whole number of shares we can buy
            shares = int(target_investment_per_stock / initial_price)
            
            if shares > 0:
                initial_cost = shares * initial_price
                final_value = shares * final_price
                
                # 2-quarter return (6 months)
                two_quarter_return = ((final_value / initial_cost) - 1) * 100
                
                # Annualized return using the formula: (1 + r)^(12/n) - 1
                # where n is number of months (6 in this case) and r is the return in decimal form
                annualized_return = (((1 + (two_quarter_return/100)) ** (12/6)) - 1) * 100
                
                # Track investment details
                portfolio_list.append({
                    'ticker': stock['ticker'],
                    'shares': shares,
                    'initial_price': initial_price,
                    'final_price': final_price,
                    'initial_cost': initial_cost,
                    'final_value': final_value,
                    'two_quarter_return_pct': two_quarter_return,  # renamed from return_pct
                    'annualized_return_pct': annualized_return,    # new field
                    'market_cap_percentile': stock['market_cap_percentile'],
                    'avg_zscore': stock['avg_zscore'],
                    'avg_zscore_rank': stock['avg_zscore_rank']
                })
                
                total_invested += initial_cost
                total_final_value += final_value
                remaining_cash = remaining_cash - initial_cost
    
    # Create portfolio summary DataFrame
    portfolio_summary = pd.DataFrame(portfolio_list)
    
    # Add experiment_id to portfolio summary
    portfolio_summary['experiment_id'] = experiment_id
    
    # Calculate aggregate performance metrics
    performance_metrics = {
        # Experiment ID
        'experiment_id': experiment_id,
        
        # Experiment parameters
        'market_cap_percentile': market_cap_percentile,
        'initial_investment': initial_investment,
        'split_method': split_method,
        'stocks_to_invest': stocks_to_invest,
        'min_zscore_rank': min_zscore_rank,
        
        # Performance metrics
        'total_stocks': len(portfolio_list),
        'total_invested': total_invested,
        'total_final_value': total_final_value,
        'remaining_cash': remaining_cash,

        # Total portfolio returns
        'total_two_quarter_return_pct': ((total_final_value / total_invested) - 1) * 100 if total_invested > 0 else 0,
        'total_annualized_return_pct': (((1 + ((total_final_value / total_invested) - 1)) ** (12/6)) - 1) * 100 if total_invested > 0 else 0,
    
        # Average of individual stock returns
        'avg_two_quarter_return_pct': portfolio_summary['two_quarter_return_pct'].mean() if len(portfolio_list) > 0 else 0,
        'avg_annualized_return_pct': portfolio_summary['annualized_return_pct'].mean() if len(portfolio_list) > 0 else 0,
        
        # Add weighted average returns (this would be more accurate)
        'weighted_avg_two_quarter_return': ((portfolio_summary['two_quarter_return_pct'] * portfolio_summary['initial_cost']).sum() / total_invested) if total_invested > 0 else 0,
        'weighted_avg_annualized_return': ((portfolio_summary['annualized_return_pct'] * portfolio_summary['initial_cost']).sum() / total_invested) if total_invested > 0 else 0,

        # Add Sortino ratio (both for individual stocks and portfolio)
        'portfolio_sortino_ratio': calculate_sortino_ratio(portfolio_summary['two_quarter_return_pct']),
        'avg_stock_sortino_ratio': np.mean([calculate_sortino_ratio([r]) for r in portfolio_summary['two_quarter_return_pct']]),
        
        # Add other useful portfolio metrics
        'max_drawdown_pct': ((portfolio_summary['two_quarter_return_pct'].min()) if len(portfolio_list) > 0 else 0),
        'positive_returns_pct': (len(portfolio_summary[portfolio_summary['two_quarter_return_pct'] > 0]) / len(portfolio_summary) * 100) if len(portfolio_list) > 0 else 0,
        'return_volatility': portfolio_summary['two_quarter_return_pct'].std() if len(portfolio_list) > 0 else 0,
        'win_loss_ratio': (len(portfolio_summary[portfolio_summary['two_quarter_return_pct'] > 0]) / 
                          len(portfolio_summary[portfolio_summary['two_quarter_return_pct'] < 0])) if len(portfolio_summary[portfolio_summary['two_quarter_return_pct'] < 0]) > 0 else float('inf')
        }
    
    # Save individual backtest results
    #create backtest folder if it doesn't exist
    if not os.path.exists('backtest'):
        os.makedirs('backtest')
    portfolio_summary.to_csv(f'backtest/operations_exp_{experiment_id}.csv', index=False)
    
    # Save experiment results
    experiment_df = pd.DataFrame([performance_metrics])
    try:
        #If file exists, append new experiment
        existing_experiments = pd.read_csv('experiments.csv')
        updated_experiments = pd.concat([existing_experiments, experiment_df], ignore_index=True)
        
    except FileNotFoundError:
        # If file doesn't exist, create new one
        updated_experiments = experiment_df
    
    # Save updated experiments
    updated_experiments.to_csv('experiments.csv', index=False)
    
    
    # Print summary with better organization
    print("\nPortfolio Performance Summary:")
    print(f"Parameters: {market_cap_percentile}, {initial_investment}, {split_method}, {stocks_to_invest}, {min_zscore_rank}")
    
    print("Portfolio Size and Value:")
    print(f"Number of stocks in portfolio: {performance_metrics['total_stocks']}")
    print(f"Total amount invested: ${performance_metrics['total_invested']:,.2f}")
    print(f"Final portfolio value: ${performance_metrics['total_final_value']:,.2f}")
    print(f"Remaining cash: ${performance_metrics['remaining_cash']:,.2f}")
    
    print("\nReturn Metrics:")
    print("Total Portfolio Returns:")
    print(f"• Total 2-quarter return: {performance_metrics['total_two_quarter_return_pct']:.2f}%")
    print(f"• Total annualized return: {performance_metrics['total_annualized_return_pct']:.2f}%")
    
    print("\nAverage Stock Returns:")
    print(f"• Simple average 2-quarter return: {performance_metrics['avg_two_quarter_return_pct']:.2f}%")
    print(f"• Simple average annualized return: {performance_metrics['avg_annualized_return_pct']:.2f}%")
    
    print("\nWeighted Average Returns:")
    print(f"• Weighted average 2-quarter return: {performance_metrics['weighted_avg_two_quarter_return']:.2f}%")
    print(f"• Weighted average annualized return: {performance_metrics['weighted_avg_annualized_return']:.2f}%")
    
    print("\nRisk Metrics:")
    print(f"• Portfolio Sortino Ratio: {performance_metrics['portfolio_sortino_ratio']:.2f}")
    print(f"• Return Volatility: {performance_metrics['return_volatility']:.2f}%")
    print(f"• Maximum Drawdown: {performance_metrics['max_drawdown_pct']:.2f}%")
    
    print("\nSuccess Metrics:")
    print(f"• Win/Loss Ratio: {performance_metrics['win_loss_ratio']:.2f}")
    print(f"• Positive Returns %: {performance_metrics['positive_returns_pct']:.2f}%")
    
    print("-" * 50)
    return None

## Running the experiment with 1000 different portfolio configurations ##

In [32]:
""" Commented so it doesnt run the experiments again
# Define parameter ranges for grid search
grid_search = {
    'market_cap_percentile': [0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50],
    'initial_investment': [10000],
    'split_method': ['equal'],
    'stocks_to_invest': [5,10, 15, 20,25, 30, 35, 40, 45, 50],
    'min_zscore_rank': [5, 10, 15, 20,25, 30, 35, 40, 45, 50]
}

# Create all combinations of parameters
param_combinations = [dict(zip(grid_search.keys(), v)) for v in itertools.product(*grid_search.values())]

# Run backtest for each parameter combination
for i, params in enumerate(param_combinations):
    print(f"\nRunning backtest with parameters: {params}") 
    strategy_backtest(
        data=results,
        market_cap_percentile=params['market_cap_percentile'],
        initial_investment=params['initial_investment'], 
        split_method=params['split_method'],
        stocks_to_invest=params['stocks_to_invest'],
        min_zscore_rank=params['min_zscore_rank'],
        experiment_id= i+1
    )
    print('='*100)
"""

' Commented so it doesnt run the experiments again\n# Define parameter ranges for grid search\ngrid_search = {\n    \'market_cap_percentile\': [0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50],\n    \'initial_investment\': [10000],\n    \'split_method\': [\'equal\'],\n    \'stocks_to_invest\': [5,10, 15, 20,25, 30, 35, 40, 45, 50],\n    \'min_zscore_rank\': [5, 10, 15, 20,25, 30, 35, 40, 45, 50]\n}\n\n# Create all combinations of parameters\nparam_combinations = [dict(zip(grid_search.keys(), v)) for v in itertools.product(*grid_search.values())]\n\n# Run backtest for each parameter combination\nfor i, params in enumerate(param_combinations):\n    print(f"\nRunning backtest with parameters: {params}") \n    strategy_backtest(\n        data=results,\n        market_cap_percentile=params[\'market_cap_percentile\'],\n        initial_investment=params[\'initial_investment\'], \n        split_method=params[\'split_method\'],\n        stocks_to_invest=params[\'stocks_to_invest\'],\

## Analyzing results of 1000 experiments ##

In [2]:
experiments = pd.read_csv('experiments.csv')

In [3]:
experiments.sort_values(by='total_annualized_return_pct', ascending=False)

,experiment_id,market_cap_percentile,initial_investment,split_method,stocks_to_invest,min_zscore_rank,total_stocks,total_invested,total_final_value,remaining_cash,...,avg_two_quarter_return_pct,avg_annualized_return_pct,weighted_avg_two_quarter_return,weighted_avg_annualized_return,portfolio_sortino_ratio,avg_stock_sortino_ratio,max_drawdown_pct,positive_returns_pct,return_volatility,win_loss_ratio
0,1,0.05,10000,equal,5,5,1,9998.549767,21868.288879,1.450233,...,118.714608,378.360795,118.714608,378.360795,0.000000,0.000000,118.714608,100.000000,NaN,inf
41,42,0.05,10000,equal,25,10,1,9998.549767,21868.288879,1.450233,...,118.714608,378.360795,118.714608,378.360795,0.000000,0.000000,118.714608,100.000000,NaN,inf
21,22,0.05,10000,equal,15,10,1,9998.549767,21868.288879,1.450233,...,118.714608,378.360795,118.714608,378.360795,0.000000,0.000000,118.714608,100.000000,NaN,inf
30,31,0.05,10000,equal,20,5,1,9998.549767,21868.288879,1.450233,...,118.714608,378.360795,118.714608,378.360795,0.000000,0.000000,118.714608,100.000000,NaN,inf
31,32,0.05,10000,equal,20,10,1,9998.549767,21868.288879,1.450233,...,118.714608,378.360795,118.714608,378.360795,0.000000,0.000000,118.714608,100.000000,NaN,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,700,0.35,10000,equal,50,50,24,8900.735109,11336.019350,1099.264891,...,26.593585,76.452006,27.360484,78.251713,2.585326,-0.471405,-50.467292,66.666667,41.105569,2.0
659,660,0.35,10000,equal,30,50,24,8900.735109,11336.019350,1099.264891,...,26.593585,76.452006,27.360484,78.251713,2.585326,-0.471405,-50.467292,66.666667,41.105569,2.0
669,670,0.35,10000,equal,35,50,24,8900.735109,11336.019350,1099.264891,...,26.593585,76.452006,27.360484,78.251713,2.585326,-0.471405,-50.467292,66.666667,41.105569,2.0
689,690,0.35,10000,equal,45,50,24,8900.735109,11336.019350,1099.264891,...,26.593585,76.452006,27.360484,78.251713,2.585326,-0.471405,-50.467292,66.666667,41.105569,2.0


In [31]:
import plotly.express as px

fig = px.scatter(
    experiments,
    x='total_annualized_return_pct',
    y='portfolio_sortino_ratio',
    title='Annualized Returns vs Sortino Ratio',
    labels={
        'total_annualized_return_pct': 'Annualized Return (%)',
        'sortino_ratio': 'Sortino Ratio',
        'total_stocks': 'Number of Stocks'
    },
    hover_data=['experiment_id', 'market_cap_percentile', 'total_stocks', 'min_zscore_rank']
)

fig.update_layout(
    xaxis_title="Annualized Return (%)",
    yaxis_title="Sortino Ratio",
    template="plotly_white"
)

fig.show()


In [29]:
import plotly.express as px
import numpy as np

# Create pivot tables for heatmap
pivot_returns = experiments.pivot_table(
    values='total_annualized_return_pct',
    index='market_cap_percentile',
    columns='min_zscore_rank',
    aggfunc='median'
)

pivot_sortino = experiments.pivot_table(
    values='portfolio_sortino_ratio', 
    index='market_cap_percentile',
    columns='min_zscore_rank',
    aggfunc='median'
)

pivot_stocks = experiments.pivot_table(
    values='total_stocks',
    index='market_cap_percentile', 
    columns='min_zscore_rank',
    aggfunc='mean'
)

# Format labels to include all metrics
labels = []
for i in range(len(pivot_returns.values)):
    for j in range(len(pivot_returns.values[0])):
        label = f"Median Return: {pivot_returns.values[i][j]:.1f}%<br>Median Sortino: {pivot_sortino.values[i][j]:.2f}<br>Median Stocks: {pivot_stocks.values[i][j]:.0f}"
        labels.append(label)

fig = px.imshow(
    pivot_returns,
    title='Returns by Market Cap & Z-Score Rank',
    labels={
        'x': 'Minimum Z-Score Rank',
        'y': 'Market Cap Percentile',
        'color': 'Median Annualized Return (%)'
    },
    color_continuous_scale='RdYlGn',
    aspect='auto'
)

fig.update_layout(
    template="plotly_white",
    xaxis_title='Minimum Z-Score Rank',
    yaxis_title='Market Cap Percentile',
    height=600  # Make the plot taller to give more room for text
)

# Add hover data after creating the figure
fig.data[0].customdata = np.array(labels).reshape(pivot_returns.shape)
fig.data[0].hovertemplate = "Market Cap Percentile: %{y}<br>Min Z-Score Rank: %{x}<br>%{customdata}<extra></extra>"

fig.show()

## Insights ##

- Main problem is concentration given that experiments with highest returns are picky in the quality of the stock and therefore pick few of them
- However we can reduce the min z_score_rank(which basically evaluates quality of the stock) to be less exclusive
- In the first chart we can see that doing that doesn't affect returns cause relaxing those metrics still keeps us on the 9% returns